In [1]:
from __future__ import print_function
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; } </style>"))
import matplotlib.pyplot as plt
from pkg.marker_config import *
from pkg.constraint_graph import *
from pkg.constraint.constraint_action import *
from pkg.constraint.constraint_object import *
from pkg.constants import *
from pkg.utils.plot_utils import *
from pkg.utils.utils import *
from pkg.environment_builder import *
from pkg.ui.ui_broker import *
from pkg.controller.combined_robot import *

from pkg.controller.combined_robot import CombinedRobot, XYZ_RPY_ROBOTS_DEFAULT
from pkg.data_collecting.sampling import *
import Data_2_Check
import Data_3_Convert

ROBOT_DICT = {"indy0": RobotType.indy7, "panda1": RobotType.panda}
gtimer = GlobalTimer.instance()
gtimer.reset()

In [2]:
root_dir=None
BASE_LINK="base_link"
ROBOT_NAMES=["indy0", "panda1"]
MAX_REACH_DICT = {'indy0': 1.5, 'panda1': 1.5}
REACH_OFFSET_DICT = {'indy0': (0, 0, 0.3), 'panda1': (0, 0, 0.3)}
GRIPPER_REFS = {"indy0": {"link_name": "indy0_tcp", "tcp_ref": [0, 0, 0.14], "depth_range": (-0.07, 0.00),
                       "width_range": (0.03, 0.05), "bname": "grip0"},
             "panda1": {"link_name": "panda1_hand", "tcp_ref": [0, 0, 0.112], "depth_range": (-0.04, 0.00),
                        "width_range": (0.03, 0.06), "bname": "grip1"}
             }
WDH = (3, 3, 3)
WDH_MIN_RBT = (0.75, 0.75, 0.5)
WDH_MAX_RBT = (2.25, 2.25, 1.0)
L_CELL = 0.2
MIN_DIST_ROBOT = 1
NoMin = 30
NoMax = 60
RATIO_COVER = 2.0
RATIO_DIMS = 2.0
LINK_COLL_MARGIN = 0.01
N_search = 10
N_retry = 1
GLOBAL_FILENAME = "global.json"
WORLD_FILENAME = "world.json"
SCENE_FILENAME = "scene.json"
SAMPLE_NUM_WORLD = 1
SAMPLE_NUM_SCENE = 1
SAMPLE_NUM_ACTION = 1
S_F_RATIO = 3
TIMEOUT = 1.0
Trbt_dict_fixed = None

In [3]:

Nrbt = len(ROBOT_NAMES) # 2
crob = CombinedRobot(robots_on_scene=[(rname, ROBOT_DICT[rname]) for rname in ROBOT_NAMES], connection_list=[False]*Nrbt)
Njoints = len(crob.joint_names),  # 13

VISUALIZE = True
graph = None
DATA_PATH = "./data" if root_dir is None else root_dir
try: os.mkdir(DATA_PATH)
except: pass

DATASET = get_now()
CURRENT_PATH = os.path.join(DATA_PATH,DATASET)
os.mkdir(CURRENT_PATH)

SAMPLED_DATA = defaultdict(dict)
GLOBAL_PARAMS = {}
__local_params_bak = __local_params = None
__local_params_bak = list(locals().keys())
####################################################################################
######################## Global parameter section ##################################
######################## Global parameter section ##################################
####################################################################################
__local_params = locals()
for k in ["BASE_LINK", "ROBOT_NAMES", "MAX_REACH_DICT", "REACH_OFFSET_DICT", "GRIPPER_REFS",
            "WDH", "WDH_MIN_RBT", "WDH_MAX_RBT", "L_CELL", "MIN_DIST_ROBOT", "NoMin", "NoMax",
            "RATIO_COVER", "RATIO_DIMS", "LINK_COLL_MARGIN","N_search", "N_retry",
            "GLOBAL_FILENAME", "WORLD_FILENAME", "SCENE_FILENAME", "SAMPLE_NUM_WORLD",
            "SAMPLE_NUM_SCENE", "SAMPLE_NUM_ACTION", "S_F_RATIO", "TIMEOUT"]:
    GLOBAL_PARAMS[k] = locals()[k]
save_json(os.path.join(CURRENT_PATH, GLOBAL_FILENAME), GLOBAL_PARAMS)


CENTER = tuple(np.divide(WDH, 2, dtype=np.float))
Ws, Ds, Hs = WDH
Nw, Nd, Nh = Nwdh = int(Ws/L_CELL), int(Ds/L_CELL), int(Hs/L_CELL)
L_MAX = L_CELL*RATIO_DIMS
print("scene size: {} ({},{},{})".format(Nw*Nd*Nh, Nw, Nd, Nh))


connection_list
[False, False]
scene size: 3375 (15,15,15)


In [4]:
# ####################################################################################
# ############################## Sample world ########################################
# ####################################################################################
# for _ in range(SAMPLE_NUM_WORLD):

In [5]:

if Trbt_dict_fixed is None:
    SAMPLED_DATA["WORLD"]["Trbt_dict"] = Trbt_dict= sample_Trbt(Nrbt, crob.robot_names, WDH_MIN_RBT, WDH_MAX_RBT, MIN_DIST_ROBOT)
else:
    SAMPLED_DATA["WORLD"]["Trbt_dict"] = Trbt_dict= Trbt_dict_fixed
reach_center_dict = {k: tuple(np.add(v[0], REACH_OFFSET_DICT[k])) for k, v in Trbt_dict.items()}
WORLD_PATH = os.path.join(CURRENT_PATH, "WORLD-"+get_now())
os.mkdir(WORLD_PATH)
save_json(os.path.join(WORLD_PATH, WORLD_FILENAME), SAMPLED_DATA["WORLD"])

cam = None
# set urdf
xcustom, JOINT_NAMES, LINK_NAMES, urdf_content = set_custom_robots(crob.robots_on_scene, Trbt_dict, crob.custom_limits, start_rviz=VISUALIZE)
ghnd = GeometryHandle(urdf_content)
time.sleep(2)


# set graph
graph = ConstraintGraph(ghnd=ghnd, urdf_path=URDF_PATH, joint_names=JOINT_NAMES, link_names=LINK_NAMES,
                        urdf_content=urdf_content, combined_robot=crob)
graph.set_camera(cam)
graph.set_cam_robot_collision(_add_cam_poles=False, color=(1,1,0,0.3))
if VISUALIZE: graph.set_rviz()

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(graph)
ui_broker.start_server()

# set rviz
if VISUALIZE: graph.set_rviz(crob.home_pose)
ui_broker.set_tables()

for gripper in GRIPPER_REFS.values():
    graph.register_binder(name=gripper['bname'], _type=FramedTool, point=gripper['tcp_ref'], rpy=[0,0,0], link_name=gripper['link_name'])
graph.register_binder(name='base', _type=PlaceFrame, point=[0,0,0], rpy=[0,0,0], link_name=BASE_LINK)
vtem = graph.ghnd.create_safe(name="virtual", gtype=GEOTYPE.SPHERE, link_name=BASE_LINK,
                              dims=(0,0,0), center=(0,0,0), rpy=(0,0,0), collision=False, display=False
                              )
graph.add_object("virtual",
                 SingleHandleObject(_object=vtem,
                                    action_point=FramePoint(name="point", _object=vtem, point=(0,0,0), rpy=(0,0,0), name_full=None)),
                 binding=("point", "base"))

obj_list=[]
col_obj_list=[]


Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [6]:
# ####################################################################################
# ############################## Sample scene ########################################
# ####################################################################################
# for _ in range(SAMPLE_NUM_SCENE):

In [7]:

SAMPLED_DATA["SCENE"]["Q_s"], _, _, _, _ = Q_s, links, link_verts, link_ctems, link_rads = sample_joint(graph)
SAMPLED_DATA["SCENE"]["Q_s_dict"] = Q_s_dict = list2dict(Q_s, crob.joint_names)

if VISUALIZE:
    graph.show_pose(Q_s)
    time.sleep(1)
    graph.show_pose(Q_s)
    time.sleep(1)
    graph.show_pose(Q_s)

gtimer.tic("extract_available")
# coll_boxes = get_colliding_cells(Nwdh, L_CELL, link_ctems, link_rads)
free_boxes = get_reachable_cells(Nwdh, L_CELL, reach_center_dict, MAX_REACH_DICT)
gtimer.toc("extract_available")

# draw_cells(graph, "col_cell", coll_boxes, L_CELL, color=(0.9, 0, 0.2, 0.1))
# remove_geometries_by_prefix(graph, "col_cell")
# draw_cells(graph, "reachable", free_boxes, L_CELL, color=(0.2, 0.7, 0.2, 0.1))
# remove_geometries_by_prefix(graph, "reachable")

link collision detected: indy0_link1_Cylinder_0 - indy0_link4_Cylinder_0
link collision detected: panda1_link5_Cylinder_0 - panda1_hand_Cylinder_0


2.164125442504883

In [8]:
obj_dat = []
for geo_gen in OBJ_GEN_LIST:
    obj_boxes = random.sample(free_boxes,int(random.uniform(NoMin, NoMax)))
    #     draw_cells(graph, "obj_cell", obj_boxes, L_CELL, color=(0.2, 0.2, 0.7, 0.1))
    #     remove_geometries_by_prefix(graph, "obj_cell")
    for nbox in obj_boxes:
        gtype, dims, color = geo_gen(L_MAX)
        #         if gtype.name == "BOX":
        #             nbox_bak = nbox
        #         else:
        #             nbox = nbox_bak
        obj_dat.append({"nbox": nbox, "gtype": gtype.name, "dims": dims, "color": color,
                        "center": tuple(np.multiply(nbox, L_CELL)+np.random.random((3,))*L_MAX+0.5*L_CELL - 0.5*L_MAX),
                        "rpy": np.random.random((3,))*np.pi*2})


# ## exclude colliding object

# In[22]:


__obj_dat=[]
coll_list = []
for odat in obj_dat:
    gtype, center, rpy, dims = getattr(GEOTYPE,odat['gtype']), odat['center'], odat['rpy'], odat['dims']
    ctem = getPointList(get_vertex_rows(gtype, center, rpy, dims))
    crad = (dims[0]/2) if gtype in [GEOTYPE.CAPSULE, GEOTYPE.CYLINDER, GEOTYPE.SPHERE] else 0
    if any(np.array(get_distance_list(ctem, link_ctems, crad, link_rads))<LINK_COLL_MARGIN):
        odat['color'] = (0.9, 0.0, 0.0, 0.3)
        coll_list.append(odat)
    else:
        __obj_dat.append(odat)
SAMPLED_DATA["SCENE"]["obj_dat"] = obj_dat = __obj_dat

SCENE_PATH = os.path.join(WORLD_PATH, "SCENE-"+get_now())
os.mkdir(SCENE_PATH)
save_json(os.path.join(SCENE_PATH, SCENE_FILENAME), SAMPLED_DATA["SCENE"])


for obj in obj_list: graph.remove_geometry(obj)
for odat in obj_dat:
    nbox, gtype, dims, color, center, rpy = odat["nbox"], getattr(GEOTYPE, odat["gtype"]), odat["dims"], odat["color"], odat["center"], odat["rpy"]
    obj = graph.ghnd.create_safe(
        name="{}_{}_{}_{}".format(gtype.name,*nbox), link_name=BASE_LINK, gtype=gtype,
        center=center, rpy=rpy, dims=dims, color=color, display=True, collision=True, fixed=True)
    obj_list.append(obj)
    graph.add_marker(obj, vis=VISUALIZE)

for obj in col_obj_list: graph.remove_geometry(obj)
for odat in coll_list:
    nbox, gtype, dims, color, center, rpy = odat["nbox"], getattr(GEOTYPE, odat["gtype"]), odat["dims"], odat["color"], odat["center"], odat["rpy"]
    obj = graph.ghnd.create_safe(
        name="col_obj_{}_{}_{}".format(*nbox), link_name=BASE_LINK, gtype=gtype,
        center=center, rpy=rpy, dims=dims, color=color, display=True, collision=False, fixed=True)
    col_obj_list.append(obj)
    graph.add_marker(obj, vis=VISUALIZE)

for obj in col_obj_list: graph.remove_geometry(obj)

if VISUALIZE: graph.set_rviz()

In [ ]:


####################################################################################
############################## Sample action #######################################
####################################################################################
for _ in range(SAMPLE_NUM_ACTION):
    dcol = DataCollector(graph, GRIPPER_REFS, S_F_RATIO=S_F_RATIO)
    if VISUALIZE: graph.set_rviz()

    # planners
    mplan = MoveitPlanner(joint_names=graph.joint_names, link_names=graph.link_names, urdf_path=graph.urdf_path, urdf_content=graph.urdf_content,
                          robot_names=graph.combined_robot.robot_names, binder_links=[v.object.link_name for v in graph.binder_dict.values()], ghnd=graph.ghnd)
    dual_mplan_dict = get_dual_planner_dict(GRIPPER_REFS, graph.ghnd, graph.urdf_content, graph.urdf_path, graph.link_names, graph.combined_robot.robot_names)

    # handover
    dcol.search_loop_mp(Q_s, obj_list, dual_mplan_dict, search_fun=dcol.handover_search, L_CELL=L_CELL, N_agents=None, N_search=N_search, N_retry=N_retry, timeout=TIMEOUT)
    save_json(os.path.join(SCENE_PATH, get_now()+".json"),  {idx: {k:v for k,v in item.items() if k !="trajectory"} for idx, item in dcol.snode_dict.items()})
    if VISUALIZE: dcol.play_all(graph, GRIPPER_REFS, "HANDOVER", test_pick, Q_s, remove_map=[[],[0,1]])

    # pick
    graph.set_planner(mplan)
    mplan.update(graph)
    dcol.search_loop_mp(Q_s, obj_list, mplan, search_fun=dcol.pick_search, L_CELL=L_CELL, N_agents=None, timeout=TIMEOUT, N_search=N_search, N_retry=N_retry)
    save_json(os.path.join(SCENE_PATH, get_now()+".json"),  {idx: {k:v for k,v in item.items() if k !="trajectory"} for idx, item in dcol.snode_dict.items()})
    if VISUALIZE: dcol.play_all(graph, GRIPPER_REFS, "PICK", test_pick, Q_s, remove_map=[[1],[0]])

    #place
    graph.set_planner(mplan)
    mplan.update(graph)
    dcol.search_loop_mp(Q_s, obj_list, mplan, search_fun=dcol.place_search, L_CELL=L_CELL, N_agents=None, timeout=TIMEOUT, N_search=N_search, N_retry=N_retry)
    save_json(os.path.join(SCENE_PATH, get_now()+".json"),  {idx: {k:v for k,v in item.items() if k !="trajectory"} for idx, item in dcol.snode_dict.items()})
    if VISUALIZE: dcol.play_all(graph, GRIPPER_REFS, "PLACE", test_place, Q_s, remove_map=[[],[0,1]])



Use 20/20 agents
1: FAILURE - 0
9: FAILURE - 0
9: FAILURE - 1
3: FAILURE - 0
5: FAILURE - 0
0: FAILURE - 0
4: FAILURE - 0
1: FAILURE - 1
2: FAILURE - 0
6: FAILURE - 0
10: FAILURE - 0
8: FAILURE - 0
7: FAILURE - 0
11: FAILURE - 0
15: FAILURE - 0
13: FAILURE - 0
14: FAILURE - 0
16: FAILURE - 0
12: FAILURE - 0
17: FAILURE - 0
18: FAILURE - 0
9: FAILURE - 2
19: FAILURE - 0
3: FAILURE - 1
5: FAILURE - 1
0: FAILURE - 1
2: FAILURE - 1
4: FAILURE - 1
10: FAILURE - 1
7: FAILURE - 1
1: FAILURE - 2
8: FAILURE - 1
6: FAILURE - 1
15: FAILURE - 1
11: FAILURE - 1
13: FAILURE - 1
16: FAILURE - 1
14: FAILURE - 1
18: FAILURE - 1
12: FAILURE - 1
17: FAILURE - 1
9: FAILURE - 3
19: FAILURE - 1
9: SUCCESS - 4
=========== indy0-panda1 9 SUCCESS =========== - 0
3: FAILURE - 2
0: FAILURE - 2
5: FAILURE - 2
2: FAILURE - 2
1: FAILURE - 3
10: FAILURE - 2
13: FAILURE - 2
6: FAILURE - 2
4: FAILURE - 2
7: FAILURE - 2
15: FAILURE - 2
8: FAILURE - 2
11: FAILURE - 2
14: FAILURE - 2
5: FAILURE - 3
12: FAILURE - 2
18: FA

11: FAILURE - 7
=========== panda1 11 FAILURE =========== - 8
13: FAILURE - 7
=========== indy0 13 FAILURE =========== - 9
1: FAILURE - 7
0: FAILURE - 8
=========== indy0 0 FAILURE =========== - 10
2: FAILURE - 7
10: FAILURE - 7
9: FAILURE - 7
5: FAILURE - 7
4: FAILURE - 7
16: FAILURE - 7
6: FAILURE - 7
19: FAILURE - 7
12: FAILURE - 7
7: FAILURE - 7
14: FAILURE - 7
15: FAILURE - 7
18: FAILURE - 7
17: FAILURE - 7
8: SUCCESS - 7
=========== panda1 8 SUCCESS =========== - 11
3: FAILURE - 7
=========== indy0 3 FAILURE =========== - 12
14: SUCCESS - 8
=========== indy0 14 SUCCESS =========== - 13
11: FAILURE - 8
13: FAILURE - 8
=========== panda1 13 FAILURE =========== - 14
1: FAILURE - 8
9: FAILURE - 8
2: FAILURE - 8
10: FAILURE - 8
4: FAILURE - 8
16: FAILURE - 8
5: FAILURE - 8
0: FAILURE - 9
=========== panda1 0 FAILURE =========== - 15
6: FAILURE - 8
=============== TERMINATE 0 ==============
19: FAILURE - 8
12: FAILURE - 8
7: FAILURE - 8
18: FAILURE - 8
17: FAILURE - 8
15: FAILURE - 8
1

In [ ]:

print("======================== ALL FINISHED ====================================")
xcustom.clear()
rospy.signal_shutdown("ALL FINISHED")

In [1]:
from __future__ import print_function
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; } </style>"))
import matplotlib.pyplot as plt
from pkg.marker_config import *
from pkg.constraint_graph import *
from pkg.constraint.constraint_action import *
from pkg.constraint.constraint_object import *
from pkg.constants import *
from pkg.utils.plot_utils import *
from pkg.utils.utils import *
from pkg.environment_builder import *
from pkg.ui.ui_broker import *
from pkg.controller.combined_robot import *

gtimer = GlobalTimer.instance()
gtimer.reset()

ImportError: /home/tamp/Projects/rnb-planning/lib/moveit_py_interface/moveit_py_interface.so: undefined symbol: _ZTIN4ompl4base10ConstraintE

In [ ]:
from pkg.controller.combined_robot import CombinedRobot, XYZ_RPY_ROBOTS_DEFAULT
from pkg.data_collecting.sampling import *
crob = CombinedRobot(connection_list=(False, False))

In [ ]:
GLOBAL_FILENAME = "global.json"
WORLD_FILENAME = "world.json"
SCENE_FILENAME = "scene.json"
VISUALIZE = True

In [ ]:
DATA_PATH = "./data"
try: os.mkdir(DATA_PATH)
except: pass

In [ ]:
import datetime
def get_now():
    return datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

In [ ]:
CURRENT_PATH = os.path.join(DATA_PATH,get_now())
os.mkdir(CURRENT_PATH)

## Global params definition

In [2]:
GLOBAL_PARAMS = dict(
    BASE_LINK = "base_link",
    MAX_REACH_DICT = {'indy0': 1.5, 'panda1': 1.5},
    REACH_OFFSET_DICT = {'indy0': (0,0,0.3), 'panda1': (0,0,0.3)},
    GRIPPER_REFS = {"indy0":{"link_name": "indy0_tcp", "tcp_ref": [0,0,0.14], "depth_range": (-0.07,0.00), "width_range": (0.03,0.05), "bname": "grip0"}, 
                    "panda1":{"link_name": "panda1_hand", "tcp_ref": [0,0,0.112], "depth_range": (-0.04,0.00), "width_range": (0.03,0.06), "bname": "grip1"}
                   },
    Nrbt = len(crob.robot_names), # 2
    Njoints = len(crob.joint_names), # 13
    WDH = (3,3,3),
    WDH_MIN_RBT = (0.75,0.75,0.5), 
    WDH_MAX_RBT = (2.25,2.25,1.0),
    L_CELL = 0.2,
    MIN_DIST_ROBOT = 1,
    No = 50,
    RATIO_COVER = 2.0,
    RATIO_DIMS = 2.0,
    LINK_COLL_MARGIN = 0.01,
    N_retry = 1
)
SAMPLED_DATA = defaultdict(dict)

NameError: name 'crob' is not defined

In [3]:
save_json(os.path.join(CURRENT_PATH, GLOBAL_FILENAME), GLOBAL_PARAMS)

NameError: name 'CURRENT_PATH' is not defined

## set params

In [9]:
__local_params = locals()
for k,v in GLOBAL_PARAMS.items():
    __local_params[k] = v
CENTER = tuple(np.divide(WDH, 2, dtype=np.float))
Ws, Ds, Hs = WDH 
Nw, Nd, Nh = Nwdh = int(Ws/L_CELL), int(Ds/L_CELL), int(Hs/L_CELL)
L_MAX = L_CELL*RATIO_DIMS
print("scene size: {} ({},{},{})".format(Nw*Nd*Nh, Nw, Nd, Nh))

scene size: 3375 (15,15,15)


In [10]:
SAMPLED_DATA["WORLD"]["Trbt_dict"] = Trbt_dict= sample_Trbt(Nrbt, crob.robot_names, WDH_MIN_RBT, WDH_MAX_RBT, MIN_DIST_ROBOT)
reach_center_dict = {k: tuple(np.add(v[0], REACH_OFFSET_DICT[k])) for k, v in Trbt_dict.items()}

In [11]:
WORLD_PATH = os.path.join(CURRENT_PATH, "WORLD-"+get_now())
os.mkdir(WORLD_PATH)
save_json(os.path.join(WORLD_PATH, WORLD_FILENAME), SAMPLED_DATA["WORLD"])

# LEVEL: WORLD

## put robot

In [12]:
cam = None
# set urdf
xcustom, JOINT_NAMES, LINK_NAMES, urdf_content = set_custom_robots(crob.robots_on_scene, Trbt_dict, crob.custom_limits, start_rviz=VISUALIZE)
ghnd = GeometryHandle(urdf_content)
time.sleep(2)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


In [13]:
# set graph
if "graph" in locals():
    graph.clear_markers()
    graph.clear_highlight()
    graph.ghnd.clear()
    graph.__init__(ghnd=graph.ghnd, urdf_path=URDF_PATH, joint_names=JOINT_NAMES, link_names=LINK_NAMES, 
                   urdf_content=urdf_content, combined_robot=crob)
    graph.set_cam_robot_collision(_add_cam_poles=False, color=(1,1,0,0.3))
    if VISUALIZE: graph.set_rviz()
else:

    graph = ConstraintGraph(ghnd=ghnd, urdf_path=URDF_PATH, joint_names=JOINT_NAMES, link_names=LINK_NAMES, 
                            urdf_content=urdf_content, combined_robot=crob)
    graph.set_camera(cam)
    graph.set_cam_robot_collision(_add_cam_poles=False, color=(1,1,0,0.3))
    graph.set_rviz()

    # start UI
    ui_broker = UIBroker.instance()
    ui_broker.initialize(graph)
    ui_broker.start_server()

    # set rviz
    graph.set_rviz(crob.home_pose)
    ui_broker.set_tables()

Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker
Please create a subscriber to the marker
Please create a subscriber to the marker
Please create a subscriber to the marker
Please create a subscriber to the marker
Please create a subscriber to the marker
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [14]:
for gripper in GRIPPER_REFS.values():
    graph.register_binder(name=gripper['bname'], _type=FramedTool, point=gripper['tcp_ref'], rpy=[0,0,0], link_name=gripper['link_name'])
graph.register_binder(name='base', _type=PlaceFrame, point=[0,0,0], rpy=[0,0,0], link_name=BASE_LINK)
vtem = graph.ghnd.create_safe(name="virtual", gtype=GEOTYPE.SPHERE, link_name=BASE_LINK, 
                                             dims=(0,0,0), center=(0,0,0), rpy=(0,0,0), collision=False, display=False
                                            )
graph.add_object("virtual",
                 SingleHandleObject(_object=vtem,
                                   action_point=FramePoint(name="point", _object=vtem, point=(0,0,0), rpy=(0,0,0), name_full=None)), 
                 binding=("point", "base"))

## show workspace

In [15]:
# show_workspace(graph, Nwdh, CENTER, L_CELL)

In [ ]:
# remove_geometries_by_prefix(graph, "workspace")
# remove_geometries_by_prefix(graph, "grid")

# LEVEL: PREMISE

## init joints

In [17]:
from pkg.data_collecting.sampling import *

In [18]:
SAMPLED_DATA["SCENE"]["Q_s"], _, _, _, _ = Q_s, links, link_verts, link_ctems, link_rads = sample_joint(graph)
SAMPLED_DATA["SCENE"]["Q_s_dict"] = Q_s_dict = list2dict(Q_s, crob.joint_names)
        
if VISUALIZE: 
    graph.show_pose(Q_s)
    time.sleep(1)
    graph.show_pose(Q_s)
    time.sleep(1)
    graph.show_pose(Q_s)

link collision detected: indy0_link2_Cylinder_0 - indy0_tcp_Cylinder_0


## extract available cells

In [19]:
gtimer.tic("extract_available")
# coll_boxes = get_colliding_cells(Nwdh, L_CELL, link_ctems, link_rads)
free_boxes = get_reachable_cells(Nwdh, L_CELL, reach_center_dict, MAX_REACH_DICT)
gtimer.toc("extract_available")

# draw_cells(graph, "col_cell", coll_boxes, L_CELL, color=(0.9, 0, 0.2, 0.1))
# remove_geometries_by_prefix(graph, "col_cell")
# draw_cells(graph, "reachable", free_boxes, L_CELL, color=(0.2, 0.7, 0.2, 0.1))
# remove_geometries_by_prefix(graph, "reachable")

2.3109912872314453

In [20]:
# draw_cells(graph, "test", [[8,8,8]], L_CELL, color=(1, 1, 0.0, 0.3))

# LEVEL: SCENE

## sample objects

In [21]:
obj_dat = []
for geo_gen in OBJ_GEN_LIST:
    obj_boxes = random.sample(free_boxes,No)
#     draw_cells(graph, "obj_cell", obj_boxes, L_CELL, color=(0.2, 0.2, 0.7, 0.1))
#     remove_geometries_by_prefix(graph, "obj_cell")
    for nbox in obj_boxes:
        gtype, dims, color = geo_gen(L_MAX)
#         if gtype.name == "BOX":
#             nbox_bak = nbox
#         else:
#             nbox = nbox_bak
        obj_dat.append({"nbox": nbox, "gtype": gtype.name, "dims": dims, "color": color, 
                        "center": tuple(np.multiply(nbox, L_CELL)+np.random.random((3,))*L_MAX+0.5*L_CELL - 0.5*L_MAX), 
                        "rpy": np.random.random((3,))*np.pi*2})

## exclude colliding object

In [22]:
__obj_dat=[]
coll_list = []
for odat in obj_dat:
    gtype, center, rpy, dims = getattr(GEOTYPE,odat['gtype']), odat['center'], odat['rpy'], odat['dims']
    ctem = getPointList(get_vertex_rows(gtype, center, rpy, dims))
    crad = (dims[0]/2) if gtype in [GEOTYPE.CAPSULE, GEOTYPE.CYLINDER, GEOTYPE.SPHERE] else 0
    if any(np.array(get_distance_list(ctem, link_ctems, crad, link_rads))<LINK_COLL_MARGIN):
        odat['color'] = (0.9, 0.0, 0.0, 0.3)
        coll_list.append(odat)
    else:
        __obj_dat.append(odat)
SAMPLED_DATA["SCENE"]["obj_dat"] = obj_dat = __obj_dat

In [23]:
SCENE_PATH = os.path.join(WORLD_PATH, "SCENE-"+get_now())
os.mkdir(SCENE_PATH)
save_json(os.path.join(SCENE_PATH, SCENE_FILENAME), SAMPLED_DATA["SCENE"])


## put objects

In [24]:
if "obj_list" in locals(): 
    for obj in obj_list: graph.remove_geometry(obj)
obj_list=[]
for odat in obj_dat:
    nbox, gtype, dims, color, center, rpy = odat["nbox"], getattr(GEOTYPE, odat["gtype"]), odat["dims"], odat["color"], odat["center"], odat["rpy"]
    obj = graph.ghnd.create_safe(
        name="{}_{}_{}_{}".format(gtype.name,*nbox), link_name=BASE_LINK, gtype=gtype,
        center=center, rpy=rpy, dims=dims, color=color, display=True, collision=True, fixed=True)
    obj_list.append(obj)
    graph.add_marker(obj, vis=VISUALIZE)

In [25]:
if "col_obj_list" in locals(): 
    for obj in col_obj_list: graph.remove_geometry(obj)
col_obj_list=[]
for odat in coll_list:
    nbox, gtype, dims, color, center, rpy = odat["nbox"], getattr(GEOTYPE, odat["gtype"]), odat["dims"], odat["color"], odat["center"], odat["rpy"]
    obj = graph.ghnd.create_safe(
        name="col_obj_{}_{}_{}".format(*nbox), link_name=BASE_LINK, gtype=gtype,
        center=center, rpy=rpy, dims=dims, color=color, display=True, collision=False, fixed=True)
    col_obj_list.append(obj)
    graph.add_marker(obj, vis=VISUALIZE)

In [26]:
if "col_obj_list" in locals(): 
    for obj in col_obj_list: graph.remove_geometry(obj)

In [27]:
if VISUALIZE: graph.set_rviz()

# LEVEL: ACTION

## DataCollector

In [28]:
class DataCollector:
    def __init__(self, graph, GRIPPER_REFS, S_F_RATIO=2.0):
        self.manager = PriorityQueueManager()
        self.manager.start()
        self.dict_lock = self.manager.Lock()
        self.graph = graph
        self.ghnd = graph.ghnd
        self.GRIPPER_REFS = GRIPPER_REFS
        self.S_F_RATIO = S_F_RATIO
        
    def pick_search(self, ID, obj_list, Q_s, mplan, L_CELL, timeout=1, N_search=100, N_retry=5):
        graph, GRIPPER_REFS = self.graph, self.GRIPPER_REFS
        fail_count = 0
        succ_count = 0
        for i in range(N_search):
            rname, inhand, obj, _, dims_bak, color_bak= sample_pick(GRIPPER_REFS, obj_list, L_CELL, self.ghnd)
            for _ in range(N_retry):
                trajectory, Q_last, error, success = test_pick(graph, GRIPPER_REFS, rname, inhand, obj, None, Q_s, mplan, timeout=timeout)
                if success: break
            print("{}: {} - {}".format(ID, "SUCCESS" if success else "FAILURE", i))
            if success or fail_count < succ_count*self.S_F_RATIO:
                self.dict_lock.acquire()
                idx = self.snode_counter.value
                self.snode_dict[idx] = {
                    "rname1": rname, "obj1": gtem_to_dict(inhand),
                    "obj2": gtem_to_dict(obj), "rname2": None, "dims_bak": dims_bak, "color_bak": color_bak,
                    "success": success, "trajectory": trajectory}
                self.snode_counter.value = self.snode_counter.value+1
                self.dict_lock.release()
                if success:
                    succ_count+=1
                else:
                    fail_count+=1
                print("=========== {} {} {} =========== - {}".format(rname, ID, "SUCCESS" if success else "FAILURE", idx))
            reset_rendering(graph, "PICK", [obj], [inhand], dims_bak, color_bak, vis=False)
        print("=============== TERMINATE {} ==============".format(ID))
        
    def place_search(self, ID, obj_list, Q_s, mplan, L_CELL, timeout=1, N_search=100, N_retry=1):
        graph, GRIPPER_REFS = self.graph, self.GRIPPER_REFS
        fail_count = 0
        succ_count = 0
        for i in range(N_search):
            rname, inhand, ontarget, _, dims_bak, color_bak= sample_place(GRIPPER_REFS, obj_list, L_CELL, self.ghnd)
            for _ in range(N_retry):
                trajectory, Q_last, error, success = test_place(graph, GRIPPER_REFS, rname, inhand, ontarget, None, Q_s, mplan, timeout=timeout)
                if success: break
            print("{}: {} - {}".format(ID, "SUCCESS" if success else "FAILURE", i))
            if success or fail_count < succ_count*self.S_F_RATIO:
                self.dict_lock.acquire()
                idx = self.snode_counter.value
                self.snode_dict[idx] = {
                    "rname1": rname, "obj1": gtem_to_dict(inhand),
                    "obj2": gtem_to_dict(ontarget), "rname2": None, "dims_bak": dims_bak, "color_bak": color_bak,
                    "success": success, "trajectory": trajectory}
                self.snode_counter.value = self.snode_counter.value+1
                self.dict_lock.release()
                if success:
                    succ_count+=1
                else:
                    fail_count+=1
                print("=========== {} {} {} =========== - {}".format(rname, ID, "SUCCESS" if success else "FAILURE", idx))
            reset_rendering(graph, "PLACE", [], [ontarget, inhand], dims_bak, color_bak, vis=False)
        print("=============== TERMINATE {} ==============".format(ID))

    def handover_search(self, ID, obj_list, Q_s, mplan_dict, L_CELL, timeout=1, N_search=100, N_retry=1):
        graph, GRIPPER_REFS = self.graph, self.GRIPPER_REFS
        fail_count = 0
        succ_count = 0
        for i in range(N_search):
            src, tar = random.sample(GRIPPER_REFS.items(),2)
            mplan = mplan_dict[(src[0],tar[0])]
            src, handed, intar, tar, dims_bak, color_bak= sample_handover(src, tar, L_CELL, mplan.ghnd)
            for _ in range(N_retry):
                trajectory, Q_last, error, success = test_handover(graph, GRIPPER_REFS, src, handed, intar, tar, Q_s, mplan, timeout=timeout)
                if success: break
            print("{}: {} - {}".format(ID, "SUCCESS" if success else "FAILURE", i))
            if success or fail_count < succ_count*self.S_F_RATIO:
                self.dict_lock.acquire()
                idx = self.snode_counter.value
                self.snode_dict[idx] = {
                    "rname1": src, "obj1": gtem_to_dict(handed),
                    "obj2": gtem_to_dict(intar), "rname2": tar, "dims_bak": dims_bak, "color_bak": color_bak,
                    "success": success, "trajectory": trajectory}
                self.snode_counter.value = self.snode_counter.value+1
                self.dict_lock.release()
                if success:
                    succ_count+=1
                else:
                    fail_count+=1
                print("=========== {}-{} {} {} =========== - {}".format(src, tar, ID, "SUCCESS" if success else "FAILURE", idx))
            reset_rendering(graph, "HANDOVER", [], [handed, intar], dims_bak, color_bak, vis=False)
        print("=============== TERMINATE {} ==============".format(ID))

    @record_time
    def search_loop_mp(self, Q_s, obj_list, mplan, search_fun, L_CELL, N_agents=None, timeout=1, N_search=100, N_retry=1):
        gtimer.tic("search_loop_mp")
        if N_agents is None:
            N_agents = cpu_count()
        self.N_agents = N_agents
        print("Use {}/{} agents".format(N_agents, cpu_count()))
        self.snode_counter = self.manager.Value('i', 0)
        self.snode_dict = self.manager.dict()
        self.proc_list = [Process(
            target=search_fun,
            args=(id_agent, obj_list, Q_s, mplan, L_CELL),
            kwargs={'timeout':timeout, 'N_search': N_search, 'N_retry': N_retry}) 
                          for id_agent in range(N_agents)]
        for proc in self.proc_list:
            proc.start()

        for proc in self.proc_list:
            proc.join()
        t = gtimer.toc("search_loop_mp")
        print("================== FINISHED in {} sec. ( {} / {} ) =======================".format(round(t/1000,1), self.snode_counter.value, N_agents*N_search))
        print(self.snode_counter.value)
        
    def play_all(self, graph, GRIPPER_REFS, key, test_fun, Q_s, period=0.05, remove_map=[[1],[0]]):
        for k in range(self.snode_counter.value):
            rname, inhand, obj, tar, dims_bak, color_bak, succ, trajectory = load_manipulation_from_dict(self.snode_dict[k], graph.ghnd)
            if succ:
                show_manip_coords(graph, GRIPPER_REFS, key, rname, inhand, obj, rname2=tar)
                graph.show_motion(trajectory, period=period)
            print("DONE: {}".format(k))
            remove1 = [[inhand, obj][iii] for iii in remove_map[0]]
            remove2 = [[inhand, obj][iii] for iii in remove_map[1]]
            reset_rendering(graph, key, remove1, remove2, dims_bak, color_bak, sleep=True, vis=True)

In [29]:
dcol = DataCollector(graph, GRIPPER_REFS)

In [ ]:
if VISUALIZE: graph.set_rviz()

## handover case - make in-hand object

In [30]:
dual_mplan_dict = get_dual_planner_dict(GRIPPER_REFS, graph.ghnd, graph.urdf_content, graph.urdf_path, graph.link_names, graph.combined_robot.robot_names)

In [31]:
dcol.search_loop_mp(Q_s, obj_list, dual_mplan_dict, search_fun=dcol.handover_search, L_CELL=L_CELL, N_agents=None, N_search=20, N_retry=1)

Use 20/20 agents
1: FAILURE - 0
0: FAILURE - 0
3: FAILURE - 0
7: FAILURE - 0
4: FAILURE - 0
5: FAILURE - 0
6: FAILURE - 0
2: SUCCESS - 0
=========== indy0-panda1 2 SUCCESS =========== - 0
8: FAILURE - 0
9: FAILURE - 0
10: FAILURE - 0
11: FAILURE - 0
12: FAILURE - 0
13: FAILURE - 0
14: FAILURE - 0
15: FAILURE - 0
18: FAILURE - 0
16: FAILURE - 0
17: SUCCESS - 0
=========== indy0-panda1 17 SUCCESS =========== - 1
19: SUCCESS - 0
=========== panda1-indy0 19 SUCCESS =========== - 2
1: FAILURE - 1
3: FAILURE - 1
0: FAILURE - 1
7: FAILURE - 1
6: FAILURE - 1
5: FAILURE - 1
9: FAILURE - 1
4: FAILURE - 1
10: FAILURE - 1
8: FAILURE - 1
12: FAILURE - 1
2: FAILURE - 1
11: FAILURE - 1
=========== panda1-indy0 2 FAILURE =========== - 3
13: FAILURE - 1
14: FAILURE - 1
15: FAILURE - 1
16: FAILURE - 1
18: FAILURE - 1
17: FAILURE - 1
=========== indy0-panda1 17 FAILURE =========== - 4
19: FAILURE - 1
=========== indy0-panda1 19 FAILURE =========== - 5
3: FAILURE - 2
1: FAILURE - 2
10: FAILURE - 2
0: FAIL

6: FAILURE - 18
1: FAILURE - 18
12: FAILURE - 18
0: FAILURE - 18
=========== panda1-indy0 12 FAILURE =========== - 45
17: FAILURE - 18
3: FAILURE - 18
7: FAILURE - 18
13: FAILURE - 14
=========== indy0-panda1 13 FAILURE =========== - 46
2: FAILURE - 19
=============== TERMINATE 2 ==============
11: FAILURE - 18
14: FAILURE - 19
15: FAILURE - 19
=============== TERMINATE 14 ==============
=============== TERMINATE 15 ==============
9: FAILURE - 19
=============== TERMINATE 9 ==============
8: FAILURE - 19
=============== TERMINATE 8 ==============
5: FAILURE - 19
=============== TERMINATE 5 ==============
10: SUCCESS - 19
=========== panda1-indy0 10 SUCCESS =========== - 47
=============== TERMINATE 10 ==============
16: FAILURE - 16
18: FAILURE - 19
=============== TERMINATE 18 ==============
19: FAILURE - 17
=========== indy0-panda1 19 FAILURE =========== - 48
6: FAILURE - 19
=============== TERMINATE 6 ==============
1: SUCCESS - 19
=========== indy0-panda1 1 SUCCESS =========== - 49

In [ ]:
save_json(os.path.join(SCENE_PATH, get_now()+".json"),  {idx: {k:v for k,v in item.items() if k !="trajectory"} for idx, item in dcol.snode_dict.items()})

## Playall

In [ ]:
if VISUALIZE: dcol.play_all(graph, GRIPPER_REFS, "HANDOVER", test_pick, Q_s, remove_map=[[],[0,1]])

# moveit

In [ ]:
mplan = MoveitPlanner(joint_names=graph.joint_names, link_names=graph.link_names, urdf_path=graph.urdf_path, urdf_content=graph.urdf_content,
                      robot_names=graph.combined_robot.robot_names, binder_links=[v.object.link_name for v in graph.binder_dict.values()], ghnd=graph.ghnd)
graph.set_planner(mplan)
mplan.update(graph)

## pick case

In [ ]:
gtimer.reset()
dcol.search_loop_mp(Q_s, obj_list, mplan, search_fun=dcol.pick_search, L_CELL=L_CELL, N_agents=None, timeout=1, N_search=20, N_retry=1)

In [ ]:
save_json(os.path.join(SCENE_PATH, get_now()+".json"),  {idx: {k:v for k,v in item.items() if k !="trajectory"} for idx, item in dcol.snode_dict.items()})

## Playall

In [ ]:
if VISUALIZE: dcol.play_all(graph, GRIPPER_REFS, "PICK", test_pick, Q_s, remove_map=[[1],[0]])

## place case

In [ ]:
gtimer.reset()
dcol.search_loop_mp(Q_s, obj_list, mplan, search_fun=dcol.place_search, L_CELL=L_CELL, N_agents=None, timeout=1, N_search=20, N_retry=1)

In [ ]:
save_json(os.path.join(SCENE_PATH, get_now()+".json"),  {idx: {k:v for k,v in item.items() if k !="trajectory"} for idx, item in dcol.snode_dict.items()})

## Playall

In [ ]:
if VISUALIZE: dcol.play_all(graph, GRIPPER_REFS, "PLACE", test_place, Q_s, remove_map=[[],[0,1]])

In [ ]:
if VISUALIZE: graph.set_rviz()